In [ ]:
from collections import deque
from typing import List, Optional, Tuple, Union
from transformers.cache_utils import Cache
from transformers.activations import ACT2FN

from transformers.modeling_rope_utils import ROPE_INIT_FUNCTIONS, dynamic_rope_update
from transformers import (
    AutoTokenizer,
    DynamicCache,
    LogitsProcessorList,
    TemperatureLogitsWarper,
    TopKLogitsWarper,
    TopPLogitsWarper,
)
from torch import nn
import torch
import os
from huggingface_hub import hf_hub_download


class Qwen3Config:
    # infra
    HF_REPO_ID = "yellooot/inferd-qwen3"
    # generation
    TEMPERATURE: float = 0.6
    TOP_K: int = 20
    TOP_P: float = 0.95
    EOS_TOKEN_ID: int = 151645
    # model
    HIDDEN_SIZE: int = 1024
    NUM_HIDDEN_LAYERS: int = 28
    NUM_ATTENTION_HEADS: int = 16
    VOCAB_SIZE: int = 151936
    MAX_POSITION_EMBEDDINGS: int = 40960
    HEAD_DIM: int = 128
    INTERMEDIATE_SIZE: int = 3072
    PAD_TOKEN_ID: int = 151643
    BOS_TOKEN_ID: int = 151643
    RMS_NORM_EPS: float = 1e-6
    HIDDEN_ACT: str = "silu"
    NUM_KEY_VALUE_HEADS: int = 8
    ATTENTION_BIAS: bool = False
    ATTENTION_TYPE: str = "full_attention"
    ROPE_THETA: int = 1000000
    TORCH_DTYPE: str = "bfloat16"


class Qwen3RMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)


class Qwen3MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_size = Qwen3Config.HIDDEN_SIZE
        self.intermediate_size = Qwen3Config.INTERMEDIATE_SIZE
        self.gate_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False)
        self.up_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False)
        self.down_proj = nn.Linear(self.intermediate_size, self.hidden_size, bias=False)
        self.act_fn = ACT2FN[Qwen3Config.HIDDEN_ACT]

    def forward(self, x):
        down_proj = self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))
        return down_proj


def rotate_half(x):
    x1 = x[..., : x.shape[-1] // 2]
    x2 = x[..., x.shape[-1] // 2 :]
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(
        batch, num_key_value_heads, n_rep, slen, head_dim
    )
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)


def eager_attention_forward(
    module: nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    scaling: float,
    dropout: float = 0.0,
):
    key_states = repeat_kv(key, module.num_key_value_groups)
    value_states = repeat_kv(value, module.num_key_value_groups)

    attn_weights = torch.matmul(query, key_states.transpose(2, 3)) * scaling
    if attention_mask is not None:
        causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
        attn_weights = attn_weights + causal_mask

    attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query.dtype)
    attn_weights = nn.functional.dropout(attn_weights, p=dropout, training=module.training)
    attn_output = torch.matmul(attn_weights, value_states)
    attn_output = attn_output.transpose(1, 2).contiguous()

    return attn_output, attn_weights


class Qwen3Attention(nn.Module):
    def __init__(self, layer_idx: int):
        super().__init__()
        self.layer_idx = layer_idx
        self.head_dim = Qwen3Config.HEAD_DIM
        self.num_key_value_groups = (
            Qwen3Config.NUM_ATTENTION_HEADS // Qwen3Config.NUM_KEY_VALUE_HEADS
        )
        self.scaling = self.head_dim**-0.5
        self.is_causal = True

        self.q_proj = nn.Linear(
            Qwen3Config.HIDDEN_SIZE,
            Qwen3Config.NUM_ATTENTION_HEADS * self.head_dim,
            bias=Qwen3Config.ATTENTION_BIAS,
        )
        self.k_proj = nn.Linear(
            Qwen3Config.HIDDEN_SIZE,
            Qwen3Config.NUM_KEY_VALUE_HEADS * self.head_dim,
            bias=Qwen3Config.ATTENTION_BIAS,
        )
        self.v_proj = nn.Linear(
            Qwen3Config.HIDDEN_SIZE,
            Qwen3Config.NUM_KEY_VALUE_HEADS * self.head_dim,
            bias=Qwen3Config.ATTENTION_BIAS,
        )
        self.o_proj = nn.Linear(
            Qwen3Config.NUM_ATTENTION_HEADS * self.head_dim,
            Qwen3Config.HIDDEN_SIZE,
            bias=Qwen3Config.ATTENTION_BIAS,
        )
        self.q_norm = Qwen3RMSNorm(self.head_dim, eps=Qwen3Config.RMS_NORM_EPS)
        self.k_norm = Qwen3RMSNorm(self.head_dim, eps=Qwen3Config.RMS_NORM_EPS)

    def forward(
        self,
        hidden_states: torch.Tensor,
        position_embeddings: Tuple[torch.Tensor, torch.Tensor],
        attention_mask: Optional[torch.Tensor],
        past_key_value: Optional[Cache] = None,
        cache_position: Optional[torch.LongTensor] = None,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        input_shape = hidden_states.shape[:-1]
        hidden_shape = (*input_shape, -1, self.head_dim)

        query_states = self.q_norm(self.q_proj(hidden_states).view(hidden_shape)).transpose(1, 2)
        key_states = self.k_norm(self.k_proj(hidden_states).view(hidden_shape)).transpose(1, 2)
        value_states = self.v_proj(hidden_states).view(hidden_shape).transpose(1, 2)

        cos, sin = position_embeddings
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        if past_key_value is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}
            key_states, value_states = past_key_value.update(
                key_states, value_states, self.layer_idx, cache_kwargs
            )

        attn_output, attn_weights = eager_attention_forward(
            self,
            query_states,
            key_states,
            value_states,
            attention_mask,
            dropout=0.0,
            scaling=self.scaling,
        )

        attn_output = attn_output.reshape(*input_shape, -1).contiguous()
        attn_output = self.o_proj(attn_output)
        return attn_output, attn_weights


class Qwen3DecoderLayer(nn.Module):
    def __init__(self, layer_idx: int):
        super().__init__()
        self.hidden_size = Qwen3Config.HIDDEN_SIZE

        self.self_attn = Qwen3Attention(layer_idx)

        self.mlp = Qwen3MLP()
        self.input_layernorm = Qwen3RMSNorm(Qwen3Config.HIDDEN_SIZE, eps=Qwen3Config.RMS_NORM_EPS)
        self.post_attention_layernorm = Qwen3RMSNorm(
            Qwen3Config.HIDDEN_SIZE, eps=Qwen3Config.RMS_NORM_EPS
        )
        self.attention_type = Qwen3Config.ATTENTION_TYPE

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        past_key_value: Optional[Cache] = None,
        cache_position: Optional[torch.LongTensor] = None,
        position_embeddings: Optional[
            Tuple[torch.Tensor, torch.Tensor]
        ] = None,  # necessary, but kept here for BC
    ) -> Tuple[torch.FloatTensor, Optional[Tuple[torch.FloatTensor, torch.FloatTensor]]]:
        residual = hidden_states
        hidden_states = self.input_layernorm(hidden_states)

        hidden_states, _ = self.self_attn(
            hidden_states=hidden_states,
            position_embeddings=position_embeddings,
            attention_mask=attention_mask,
            past_key_value=past_key_value,
            cache_position=cache_position,
        )
        hidden_states = residual + hidden_states

        residual = hidden_states
        hidden_states = self.post_attention_layernorm(hidden_states)
        hidden_states = self.mlp(hidden_states)
        hidden_states = residual + hidden_states

        return hidden_states


class Qwen3RotaryEmbedding(nn.Module):
    def __init__(self, device=None):
        super().__init__()
        self.rope_type = "default"
        self.max_seq_len_cached = Qwen3Config.MAX_POSITION_EMBEDDINGS
        self.original_max_seq_len = Qwen3Config.MAX_POSITION_EMBEDDINGS

        self.rope_init_fn = ROPE_INIT_FUNCTIONS[self.rope_type]

        inv_freq, self.attention_scaling = self.rope_init_fn(
            device=device, base=Qwen3Config.ROPE_THETA, dim=Qwen3Config.HEAD_DIM
        )
        self.register_buffer("inv_freq", inv_freq, persistent=False)
        self.original_inv_freq = self.inv_freq

    @torch.no_grad()
    @dynamic_rope_update
    def forward(self, x, position_ids):
        inv_freq_expanded = (
            self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1).to(x.device)
        )
        position_ids_expanded = position_ids[:, None, :].float()

        device_type = (
            x.device.type if isinstance(x.device.type, str) and x.device.type != "mps" else "cpu"
        )
        with torch.autocast(device_type=device_type, enabled=False):
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            emb = torch.cat((freqs, freqs), dim=-1)
            cos = emb.cos() * self.attention_scaling
            sin = emb.sin() * self.attention_scaling

        return cos.to(dtype=x.dtype), sin.to(dtype=x.dtype)


class Qwen3Server(nn.Module):
    def __init__(self, start_layer: int, end_layer: int):
        super().__init__()
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        elif torch.backends.mps.is_available():
            self.device = torch.device("mps")
        else:
            self.device = torch.device("cpu")
        self.start_layer = start_layer
        self.end_layer = end_layer
        self.past_key_value = DynamicCache()
        self.local_layers = nn.ModuleList(
            [Qwen3DecoderLayer(layer_idx) for layer_idx in range(start_layer, end_layer + 1)]
        )
        self.to(self.device)
        self._load_weights()

    def _load_weights(self):
        for layer in self.local_layers:
            idx = layer.self_attn.layer_idx
            fname = f"layer_{idx:02d}.pt"

            # Загружаем файл с HF (автоматически кэшируется в ~/.cache/huggingface)
            cached_path = hf_hub_download(repo_id=Qwen3Config.HF_REPO_ID, filename=fname)

            layer_sd = torch.load(cached_path, map_location=self.device)
            layer.load_state_dict(layer_sd)

    def send(
        self,
        hidden_states,
        attention_mask,
        cache_position,
        position_embeddings,
        input_start_layer=None,
        input_end_layer=None,
    ):
        for layer in self.local_layers:
            hidden_states = layer(
                hidden_states=hidden_states,
                attention_mask=attention_mask,
                cache_position=cache_position,
                position_embeddings=position_embeddings,
                past_key_value=self.past_key_value,
            )
        return hidden_states


class Qwen3Client(nn.Module):
    def __init__(self):
        super().__init__()
        if torch.cuda.is_available():
            self.device = torch.device("cuda")
        elif torch.backends.mps.is_available():
            self.device = torch.device("mps")
        else:
            self.device = torch.device("cpu")

        self.tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

        self.embed_tokens = nn.Embedding(
            Qwen3Config.VOCAB_SIZE, Qwen3Config.HIDDEN_SIZE, Qwen3Config.PAD_TOKEN_ID
        )
        self.norm = Qwen3RMSNorm(Qwen3Config.HIDDEN_SIZE, Qwen3Config.RMS_NORM_EPS)
        self.lm_head = nn.Linear(Qwen3Config.HIDDEN_SIZE, Qwen3Config.VOCAB_SIZE, bias=False)
        self.rotary_emb = Qwen3RotaryEmbedding()
        self.to(self.device)
        self._load_weights()

        self.logit_processors = LogitsProcessorList(
            [
                TemperatureLogitsWarper(Qwen3Config.TEMPERATURE),
                TopKLogitsWarper(Qwen3Config.TOP_K),
                TopPLogitsWarper(Qwen3Config.TOP_P),
            ]
        )
        self._dummy_input_ids = torch.zeros((1, 1), dtype=torch.long, device=self.device)

    def _load_weights(self):
        for fname, target_module in [
            ("embed_tokens.pt", self.embed_tokens),
            ("norm.pt", self.norm),
            ("lm_head.pt", self.lm_head),
        ]:
            file_path = hf_hub_download(repo_id=Qwen3Config.HF_REPO_ID, filename=fname)
            state_dict = torch.load(file_path, map_location=self.device)
            target_module.load_state_dict(state_dict)

    def _choose_next(self, logits: torch.Tensor) -> torch.Tensor:
        logits = logits.clone()
        logits = self.logit_processors(self._dummy_input_ids, logits)
        probs = torch.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, num_samples=1)
        return next_token_id

    def _should_continue(self, generated_ids: List[int], max_length: Optional[int]) -> bool:
        if max_length is not None and len(generated_ids) >= max_length:
            return False

        if len(generated_ids) > 0 and generated_ids[-1] == Qwen3Config.EOS_TOKEN_ID:
            return False

        return True

    def _find_best_chain(self, known_servers):
        return deque(
            [
                Qwen3Server(0, 9),
                Qwen3Server(10, 19),
                Qwen3Server(20, Qwen3Config.NUM_HIDDEN_LAYERS - 1),
            ]
        )

    def _update_causal_mask(
        self,
        attention_mask: torch.Tensor,
        input_tensor: torch.Tensor,
        cache_position: torch.Tensor,
        past_seen_tokens: int,
    ):
        dtype = input_tensor.dtype
        sequence_length = input_tensor.shape[1]
        target_length = (
            attention_mask.shape[-1]
            if isinstance(attention_mask, torch.Tensor)
            else past_seen_tokens + sequence_length + 1
        )
        causal_mask = self._prepare_4d_causal_attention_mask_with_cache_position(
            attention_mask,
            sequence_length=sequence_length,
            target_length=target_length,
            dtype=dtype,
            cache_position=cache_position,
            batch_size=input_tensor.shape[0],
        )

        return causal_mask

    @staticmethod
    def _prepare_4d_causal_attention_mask_with_cache_position(
        attention_mask: torch.Tensor,
        sequence_length: int,
        target_length: int,
        dtype: torch.dtype,
        cache_position: torch.Tensor,
        batch_size: int,
    ):
        if attention_mask is not None and attention_mask.dim() == 4:
            causal_mask = attention_mask
        else:
            min_dtype = torch.finfo(dtype).min
            causal_mask = torch.full(
                (sequence_length, target_length),
                fill_value=min_dtype,
                dtype=dtype,
                device=cache_position.device,
            )
            diagonal_attend_mask = torch.arange(
                target_length, device=cache_position.device
            ) > cache_position.reshape(-1, 1)

            causal_mask *= diagonal_attend_mask
            causal_mask = causal_mask[None, None, :, :].expand(batch_size, 1, -1, -1)
            if attention_mask is not None:
                causal_mask = causal_mask.clone()
                if attention_mask.shape[-1] > target_length:
                    attention_mask = attention_mask[:, :target_length]
                mask_length = attention_mask.shape[-1]
                padding_mask = causal_mask[:, :, :, :mask_length] + attention_mask[
                    :, None, None, :
                ].to(causal_mask.device)
                padding_mask = padding_mask == 0
                causal_mask[:, :, :, :mask_length] = causal_mask[:, :, :, :mask_length].masked_fill(
                    padding_mask, min_dtype
                )
        return causal_mask

    def generate(self, known_servers, prompt: str, max_length: Optional[int] = None) -> str:
        chain = self._find_best_chain(known_servers)
        with torch.no_grad():
            generated_ids: List[int] = []
            messages = [{"role": "user", "content": prompt}]
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                enable_thinking=False,
            )
            all_input_ids = self.tokenizer(text, return_tensors="pt").input_ids.to(self.device)

            input_embeds = self.embed_tokens(all_input_ids)
            batch_size, seq_len = input_embeds.shape[0], input_embeds.shape[1]

            position_ids = torch.arange(seq_len, device=self.device).unsqueeze(0)

            dtype = input_embeds.dtype
            min_val = torch.finfo(dtype).min
            tril_2d = torch.tril(torch.ones(seq_len, seq_len, device=self.device, dtype=dtype))
            mask2d = (1.0 - tril_2d) * min_val
            causal_mask_4d = (
                mask2d.unsqueeze(0).unsqueeze(0).expand(batch_size, 1, seq_len, seq_len)
            )

            pos_emb = self.rotary_emb(input_embeds, position_ids)
            hidden_states = input_embeds

            for server in chain:
                hidden_states = server.send(
                    hidden_states=hidden_states,
                    attention_mask=causal_mask_4d,
                    cache_position=position_ids.squeeze(0),
                    position_embeddings=pos_emb,
                )

            hidden_last = hidden_states[:, -1, :]
            hidden_norm = self.norm(hidden_last)
            last_logits = self.lm_head(hidden_norm)

            next_token_id = self._choose_next(last_logits)
            generated_ids.append(next_token_id.item())

            while self._should_continue(generated_ids, max_length):
                single_input_ids = next_token_id
                emb = self.embed_tokens(single_input_ids)

                small_mask2d = torch.zeros((1, 1), device=self.device, dtype=dtype)
                causal_mask_single = small_mask2d.unsqueeze(0).unsqueeze(0)

                past_len = all_input_ids.shape[1] + len(generated_ids) - 1

                position_ids = torch.tensor([[past_len]], device=self.device)

                pos_emb = self.rotary_emb(emb, position_ids)

                hidden_states = emb
                for server in chain:
                    hidden_states = server.send(
                        hidden_states=hidden_states,
                        attention_mask=causal_mask_single,
                        cache_position=position_ids.squeeze(0),
                        position_embeddings=pos_emb,
                    )

                hidden_last = hidden_states[:, -1, :]
                hidden_norm = self.norm(hidden_last)
                last_logits = self.lm_head(hidden_norm)

                next_token_id = self._choose_next(last_logits)
                generated_ids.append(next_token_id.item())

                print(
                    self.tokenizer.decode(
                        [next_token_id.item()],
                        skip_special_tokens=True,
                        clean_up_tokenization_spaces=True,
                    ),
                    end="",
                    flush=True,
                )

            output_str = self.tokenizer.decode(
                generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
            return output_str


In [2]:
client = Qwen3Client()

In [3]:
client.generate(None, "I love cats!")

 love cats too! They're such amazing pets. �� They can be super playful, curious, and loving. If you're looking for a cat, I'd love to help you find one! ���

"I love cats too! They're such amazing pets. 😊 They can be super playful, curious, and loving. If you're looking for a cat, I'd love to help you find one! 🐾"